In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.01;

In [3]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [11]:
q0 = [1., 0., 0., 0., 0., 0., .05]
v0 = [0., 0., 0., 2., 0. , 0.]
N = 60

set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)

traj = BilevelTrajOpt.simulate(x0,env,Δt,N)

Restoration_Failed
Solve_Succeeded


Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded


20×61 Array{Float64,2}:
 1.0    1.0           1.0          …   1.0           1.0        
 0.0    6.24194e-41   7.4608e-41       7.77503e-40   7.89849e-40
 0.0   -1.38778e-21  -2.77556e-21     -8.18789e-20  -8.32667e-20
 0.0    0.0          -6.62869e-41     -6.82263e-35  -6.95045e-35
 0.0    0.02          0.04             0.42279       0.42279    
 0.0    2.8226e-23    5.64862e-23  …   4.61961e-17   4.61961e-17
 0.05   0.049019      0.047057         2.18226e-9    2.18205e-9 
 0.0    2.44055e-39   2.43771e-39      2.46925e-39   2.46925e-39
 0.0   -2.77556e-19  -2.77556e-19     -2.77556e-19  -2.77556e-19
 0.0   -1.32442e-38  -1.32574e-38     -2.5563e-34   -2.5563e-34 
 2.0    2.0           2.0          …  -4.16455e-22   4.57239e-23
 0.0    2.8226e-21    2.82602e-21      1.12166e-21  -1.86304e-21
 0.0   -0.0981       -0.1962          -4.12549e-11  -2.16102e-11
 0.0    0.0           0.0              0.0           0.0        
 0.0   -2.36454e-9   -4.02173e-9       0.201733      0.201732   
 

In [13]:
qs = [traj[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .25);